In [1]:
#JORNADA 31

import pandas as pd
import random
#diccionarios
from collections import defaultdict

#Con este excel se carga la clasificación cargada
#Y también caragremos la taba de los enfrentamientos que le quedan a cada equipo
#Con header=0 conseguimos que la primera fila la tome como una fila primcipal
#el index lo ponemos para que se tenga en cuenta en en el excel que los equipos son lo que manda
clasificacion = pd.read_excel('Base de datos Montecarlo.xlsx', sheet_name='clasificacion31', header=0, index_col=0)
enfrentamientos = pd.read_excel('Base de datos Montecarlo.xlsx', sheet_name='enfrentamientos31', header=0)

#Definimos el numero de simulaciones que vamos a realizar
num_simulaciones = 10000
#Añadimos un sistema de puntos para poder implementarlo en el futuro a cualquier deporte
#El orden que sigue es V,E,D
sistema_puntos = [3, 1, 0]

#Implementamops una funcion que por cada partido nos devuleva un resultado aleatorio 
# la funcion dice que si los goles local>visitante gana local y viceversa y sino se devuelve empate
def resultado(goles_local,goles_visitante):
    resultados = ['V', 'E', 'D']
    if goles_local > goles_visitante:
        return resultados[0]
    elif goles_visitante > goles_local:
        return resultados[2]
    else:
        return resultados[1]
    
#Una funcion que asigne puntos en caso de V,D,E, nos fijamos en la raid
# En esta función asiganmos 3 puntos para el equipo que gana y 0 para su rival
def asignarpuntos(resultado):
    if resultado == 'V':
        return (sistema_puntos[0],sistema_puntos[2])
    elif resultado == 'E':
        return (sistema_puntos[1],sistema_puntos[1]) 
    else:
        return (sistema_puntos[ 2],sistema_puntos[0])

#le asignamos los puntos que obtienen en los partidos a la clasificación
#Con esta entrada definimos el campeon de liga de 100000 simulaciones 
#Nos quedaremos siempre con el primero gracias a la fumcion nlargest
def resultado_campeonliga(clasificacion):
    return clasificacion.nlargest(1,'Pts.').index[0]

def resultado_Champions(clasificacion):
    return clasificacion.nlargest(4,'Pts.').index

def resultado_EuropaLeague(clasificacion):
    return clasificacion.nlargest(6,'Pts.').index[4:6]

def resultado_Descenso(clasificacion):
    return clasificacion.nsmallest(3,'Pts.').index
    
#Esta función nos devolverá un numero de goles, poniendo maximo 5 goles en local y visitante
def asignargoles():
    goles_local = random.choice(range(0,5))
    goles_visitante = random.choice(range(0,5))
    
    return (goles_local,goles_visitante)
    
#con la funcion .loc loraremos que nos devuelva primero la informacion de la fila
#con dg1 accedemos a la informacion del equipo 1 y la diferencia de goles que hay
def desempate_golaverage(nombre_equipo1, nombre_equipo2, clasificacion_actual):
    dg1 = clasificacion_actual.loc[nombre_equipo1]['DG']
    dg2 = clasificacion_actual.loc[nombre_equipo2]['DG']
#esta función devuelve el equipo que tenga mas diferencia de goles    
    if dg1 > dg2:
        return nombre_equipo1
    elif dg2 > dg1:
        return nombre_equipo2
    else:
        goles_afavor1 = clasificacion_actual.loc[nombre_equipo1]['GF']
        goles_afavor2 = clasificacion_actual.loc[nombre_equipo2]['GF']
 #esta funcion en caso de que hayan empatado a todo, tendrá prioridad el equipo con mas goles a favor       
        if goles_afavor1 > goles_afavor2:
            return nombre_equipo1
        else:
            return nombre_equipo2
    
#esta función determina un desempate en caso de que haya un empate a puntos y tendremos que acceder a la clasificacion 
#devolvera el nombre de equipo 1 o equipo 2
def desempate_directo(nombre_equipo1, nombre_equipo2, clasificacion_actual):

    #guardamos con esto en resultado tanto para la ida como la vuelta
    resultado1 = registro_partidos.at[nombre_equipo1, nombre_equipo2]
    resultado2 = registro_partidos.at[nombre_equipo2, nombre_equipo1]

    #es la suma del resultado 1 y resultado 2 para equipo 1 y 2
    resultado_global_equipo1 = resultado1[0] + resultado2[1]
    resultado_global_equipo2 = resultado2[0] + resultado1[1]

    #si resultado 1 > resultado 2 se devuelve el nombre del equipo 1 y viceversa
    #y si es igual el resultado tendremos que pasar a la siguiente linea  de desempate por gol average
    if resultado_global_equipo1 > resultado_global_equipo2:
        return nombre_equipo1
    elif resultado_global_equipo2 > resultado_global_equipo1:
        return nombre_equipo2
    else:
        return desempate_golaverage(nombre_equipo1,nombre_equipo2, clasificacion_actual)

 #esta funcion es para los equipos empatados a puntos
def equipos_empatados(clasificacion):
    equipos_empatados = []
    
#el primer equipo se va a comparar con los demas equipos, si los pts del equipo 1 son = al equipo 2 se añade a la lista
    for idx, equipo1 in enumerate (clasificacion.index):
#comparar equipo 1 con los que estan debajo de la clasificacion
        for equipo2 in clasificacion.index[idx+1:]:
#comprobacion para ver si tiene el mismo numero de puntos
            if clasificacion.loc[equipo1, 'Pts.'] == clasificacion.loc[equipo2, 'Pts.']:
                equipos_empatados.append((equipo1, equipo2))
    return equipos_empatados

#con esta funcion reordenamos la clasificacion con los cirterios de reordenacion
def reordenacion_clasificacion(clasificacion_copia, empatados):
#Resetear el indice y guardar la columna equipo
    clasificacion_copia['Equipo'] = clasificacion_copia.index
#Desempatar y reordenar la clasificacion copia
    for equipo1, equipo2 in empatados:
        ganador = desempate_directo(equipo1, equipo2, clasificacion_actual=clasificacion_copia)
        if ganador == equipo1:
            temp = clasificacion_copia.loc[equipo2].copy()
            clasificacion_copia.loc[equipo2] = clasificacion_copia.loc[equipo1]
            clasificacion_copia.loc[equipo1] = temp
        elif ganador == equipo2:
            temp = clasificacion_copia.loc[equipo1].copy()
            clasificacion_copia.loc[equipo1] = clasificacion_copia.loc[equipo2]
            clasificacion_copia.loc[equipo2] = temp

#Restaurar Equipo como el indice 
    clasificacion_copia.set_index('Equipo', inplace=True)
    return clasificacion_copia


#Creamos un dataframe de 20 filas y 20 columnas, es una matriz asimetrica
#creamos un indice tanto para las columnas como para las filas para tener el dataframe 
#el objetivo es guardar un registro de partidos
registro_partidos = pd.DataFrame(index=range(20), columns=range(20))
registro_partidos = registro_partidos.applymap(lambda x: (0, 0))
registro_partidos.index = clasificacion.index
registro_partidos.columns = clasificacion.index
    
equipos_campeones = defaultdict(int)
equipos_champions = defaultdict(int)
equipos_europaLeague = defaultdict(int)
equipos_descensos = defaultdict(int)

#a aprtir de este bucle lo que haremos sera repetir una y otra vez las simulaciones de 0 a cualquier numer de simulaciones
for _ in range(0,num_simulaciones):
    #copiamos la clasificacion orginal para hacer tantas simulaiciones como queramos
    #y tambien he hecho una copia en enfremientos
    clasificacion_copia = clasificacion.copy()
    enfrentamientos_copia = enfrentamientos.copy()

    #se ejecuta 20 veces el bucle porque hay 20 equipos
    #gracias a esto analizmos que rival le toca a cada equipo
    #calculamos goles de forma aleatoria  y asignamos los puntos correspondientes de cada equipo
    for equipo in clasificacion.index:
        rivales = enfrentamientos_copia[equipo].dropna()
    
        for rival in rivales:
            #si el rival es none lo que haremos es que continue con el siguiente partido
            #si no es None deberá asignar puntos, goles,etc
            if rival == None:
                continue
                
            goles = asignargoles() 
            #con esto accedemos al registro de los goles tanto del local como del visitante
            registro_partidos.at[equipo , rival] = goles
            resultado_equipos = resultado(goles[0],goles[1])
            puntuacion = asignarpuntos(resultado_equipos)

            #debemos añadir los puntos los partidos jugados goles a favor y goles en contra para el equipo y el rival
            #con esta funcion actualizamos la tabal tanto para el equipo que analizamos como para su rival
            clasificacion_copia.loc[equipo]['Pts.'] += puntuacion[0]
            clasificacion_copia.loc[equipo]['PJ'] += 1
            clasificacion_copia.loc[equipo]['GF'] += goles[0]
            clasificacion_copia.loc[equipo]['GC'] += goles[1]
            clasificacion_copia.loc[equipo]['DG'] += (goles[0] - goles[1])
            
            clasificacion_copia.loc[rival]['Pts.'] += puntuacion[1]
            clasificacion_copia.loc[rival]['PJ'] += 1
            clasificacion_copia.loc[rival]['GF'] += goles[1]
            clasificacion_copia.loc[rival]['GC'] += goles[0]
            clasificacion_copia.loc[rival]['DG'] += (goles[1] - goles[0])
            
            #Esta función de none la emplearemos para que no se dupliquen los partidos
            enfrentamientos_copia.loc[enfrentamientos_copia[rival] == equipo,rival] = None

    #con esta funcion tenemos la clasificacion ordenada por puntos
    clasificacion_copia = clasificacion_copia.sort_values(by='Pts.', ascending=False) 
 
    #con esta funcion una vez tenemos la clasificacion buscamos los equipos empatados a puntos y empleamos criterios de reordeacion
    empatados = equipos_empatados(clasificacion_copia)
    clasificacion_copia = reordenacion_clasificacion(clasificacion_copia, empatados)
 
    #aqui almacenamos al campón en el diccionario de campeones 
    #con esto cada vez que un equipo sale campeon se le suma uno
    campeon = resultado_campeonliga(clasificacion=clasificacion_copia)
    equipos_campeones[campeon] += 1

    champions = resultado_Champions(clasificacion=clasificacion_copia)
    for equipo_champions in champions:
        equipos_champions[equipo_champions] += 1

    europaLeague = resultado_EuropaLeague(clasificacion=clasificacion_copia)
    for equipo_europaLeague in europaLeague:
        equipos_europaLeague[equipo_europaLeague] += 1

    descenso = resultado_Descenso(clasificacion=clasificacion_copia)
    for equipo_descenso in descenso:
        equipos_descensos[equipo_descenso] += 1

  
#Convertimos todo a una cadena de texto y lo dejamos a 2 decimales de redondeo 
#Con esto imprimimos las probabilidades de los equipos
print("Probabilidades de ganar la liga\n")   
for campeon in equipos_campeones:
    print(campeon + '-' + str(round(100*(equipos_campeones[campeon]/num_simulaciones),2)) + "%")

print("Probabilidades de entrar en Champions\n")   
for champions in equipos_champions:
    print(champions + '-' + str(round(100*(equipos_champions[champions]/num_simulaciones),2)) + "%")

print("Probabilidades de entrar en Europa League\n")   
for europaLeague in equipos_europaLeague:
    print(europaLeague + '-' + str(round(100*(equipos_europaLeague[europaLeague]/num_simulaciones),2)) + "%")

print("Probabilidades de descender\n")   
for descenso in equipos_descensos:
    print(descenso + '-' + str(round(100*(equipos_descensos[descenso]/num_simulaciones),2)) + "%")

print (clasificacion_copia)

Probabilidades de ganar la liga

Real Madrid-91.43%
Barcelona-8.05%
Girona-0.5%
Atlético-0.02%
Probabilidades de entrar en Champions

Real Madrid-100.0%
Barcelona-99.94%
Girona-95.98%
Atlético-74.75%
Athletic-27.89%
R. Sociedad-1.33%
Valencia-0.07%
Betis-0.04%
Probabilidades de entrar en Europa League

Athletic-70.92%
Betis-13.43%
Girona-4.02%
Valencia-26.6%
Atlético-25.22%
R. Sociedad-58.56%
Getafe-0.4%
Las Palmas-0.16%
Osasuna-0.29%
Villarreal-0.34%
Barcelona-0.06%
Probabilidades de descender

Granada-99.18%
Almería-99.92%
Cádiz-65.81%
Rayo-5.14%
Celta-20.54%
Mallorca-5.62%
Alavés-2.82%
Sevilla-0.88%
Villarreal-0.01%
Las Palmas-0.08%
             PJ  PG  PE  PP  GF  GC  DG  Pts.
Equipo                                       
Real Madrid  38  24   6   1  77  31  46    87
Barcelona    38  21   7   3  79  48  31    82
Atlético     38  19   4   8  73  43  30    72
Girona       38  20   5   6  77  58  19    71
R. Sociedad  38  13  11   7  53  45   8    61
Athletic     38  16   9   6  60  4

In [2]:
##JORNADA 35

import pandas as pd
import random
#diccionarios
from collections import defaultdict

#Con este excel se carga la clasificación cargada
#Y también caragremos la taba de los enfrentamientos que le quedan a cada equipo
#Con header=0 conseguimos que la primera fila la tome como una fila primcipal
#el index lo ponemos para que se tenga en cuenta en en el excel que los equipos son lo que manda
clasificacion = pd.read_excel('Base de datos Montecarlo.xlsx', sheet_name='clasificacion35', header=0, index_col=0)
enfrentamientos = pd.read_excel('Base de datos Montecarlo.xlsx', sheet_name='enfrentamientos35', header=0)

#Definimos el numero de simulaciones que vamos a realizar
num_simulaciones = 10000
#Añadimos un sistema de puntos para poder implementarlo en el futuro a cualquier deporte
#El orden que sigue es V,E,D
sistema_puntos = [3, 1, 0]

#Implementamops una funcion que por cada partido nos devuleva un resultado aleatorio 
# la funcion dice que si los goles local>visitante gana local y viceversa y sino se devuelve empate
def resultado(goles_local,goles_visitante):
    resultados = ['V', 'E', 'D']
    if goles_local > goles_visitante:
        return resultados[0]
    elif goles_visitante > goles_local:
        return resultados[2]
    else:
        return resultados[1]
    
#Una funcion que asigne puntos en caso de V,D,E, nos fijamos en la raid
# En esta función asiganmos 3 puntos para el equipo que gana y 0 para su rival
def asignarpuntos(resultado):
    if resultado == 'V':
        return (sistema_puntos[0],sistema_puntos[2])
    elif resultado == 'E':
        return (sistema_puntos[1],sistema_puntos[1]) 
    else:
        return (sistema_puntos[ 2],sistema_puntos[0])

#le asignamos los puntos que obtienen en los partidos a la clasificación
#Con esta entrada definimos el campeon de liga de 100000 simulaciones 
#Nos quedaremos siempre con el primero gracias a la fumcion nlargest
def resultado_campeonliga(clasificacion):
    return clasificacion.nlargest(1,'Pts.').index[0]

def resultado_Champions(clasificacion):
    return clasificacion.nlargest(4,'Pts.').index

def resultado_EuropaLeague(clasificacion):
    return clasificacion.nlargest(6,'Pts.').index[4:6]

def resultado_Descenso(clasificacion):
    return clasificacion.nsmallest(3,'Pts.').index
    
#Esta función nos devolverá un numero de goles, poniendo maximo 5 goles en local y visitante
def asignargoles():
    goles_local = random.choice(range(0,5))
    goles_visitante = random.choice(range(0,5))
    
    return (goles_local,goles_visitante)
    
#con la funcion .loc loraremos que nos devuelva primero la informacion de la fila
#con dg1 accedemos a la informacion del equipo 1 y la diferencia de goles que hay
def desempate_golaverage(nombre_equipo1, nombre_equipo2, clasificacion_actual):
    dg1 = clasificacion_actual.loc[nombre_equipo1]['DG']
    dg2 = clasificacion_actual.loc[nombre_equipo2]['DG']
#esta función devuelve el equipo que tenga mas diferencia de goles    
    if dg1 > dg2:
        return nombre_equipo1
    elif dg2 > dg1:
        return nombre_equipo2
    else:
        goles_afavor1 = clasificacion_actual.loc[nombre_equipo1]['GF']
        goles_afavor2 = clasificacion_actual.loc[nombre_equipo2]['GF']
 #esta funcion en caso de que hayan empatado a todo, tendrá prioridad el equipo con mas goles a favor       
        if goles_afavor1 > goles_afavor2:
            return nombre_equipo1
        else:
            return nombre_equipo2
    
#esta función determina un desempate en caso de que haya un empate a puntos y tendremos que acceder a la clasificacion 
#devolvera el nombre de equipo 1 o equipo 2
def desempate_directo(nombre_equipo1, nombre_equipo2, clasificacion_actual):

    #guardamos con esto en resultado tanto para la ida como la vuelta
    resultado1 = registro_partidos.at[nombre_equipo1, nombre_equipo2]
    resultado2 = registro_partidos.at[nombre_equipo2, nombre_equipo1]

    #es la suma del resultado 1 y resultado 2 para equipo 1 y 2
    resultado_global_equipo1 = resultado1[0] + resultado2[1]
    resultado_global_equipo2 = resultado2[0] + resultado1[1]

    #si resultado 1 > resultado 2 se devuelve el nombre del equipo 1 y viceversa
    #y si es igual el resultado tendremos que pasar a la siguiente linea  de desempate por gol average
    if resultado_global_equipo1 > resultado_global_equipo2:
        return nombre_equipo1
    elif resultado_global_equipo2 > resultado_global_equipo1:
        return nombre_equipo2
    else:
        return desempate_golaverage(nombre_equipo1,nombre_equipo2, clasificacion_actual)

 #esta funcion es para los equipos empatados a puntos
def equipos_empatados(clasificacion):
    equipos_empatados = []
    
#el primer equipo se va a comparar con los demas equipos, si los pts del equipo 1 son = al equipo 2 se añade a la lista
    for idx, equipo1 in enumerate (clasificacion.index):
#comparar equipo 1 con los que estan debajo de la clasificacion
        for equipo2 in clasificacion.index[idx+1:]:
#comprobacion para ver si tiene el mismo numero de puntos
            if clasificacion.loc[equipo1, 'Pts.'] == clasificacion.loc[equipo2, 'Pts.']:
                equipos_empatados.append((equipo1, equipo2))
    return equipos_empatados

#con esta funcion reordenamos la clasificacion con los cirterios de reordenacion
def reordenacion_clasificacion(clasificacion_copia, empatados):
#Resetear el indice y guardar la columna equipo
    clasificacion_copia['Equipo'] = clasificacion_copia.index
#Desempatar y reordenar la clasificacion copia
    for equipo1, equipo2 in empatados:
        ganador = desempate_directo(equipo1, equipo2, clasificacion_actual=clasificacion_copia)
        if ganador == equipo1:
            temp = clasificacion_copia.loc[equipo2].copy()
            clasificacion_copia.loc[equipo2] = clasificacion_copia.loc[equipo1]
            clasificacion_copia.loc[equipo1] = temp
        elif ganador == equipo2:
            temp = clasificacion_copia.loc[equipo1].copy()
            clasificacion_copia.loc[equipo1] = clasificacion_copia.loc[equipo2]
            clasificacion_copia.loc[equipo2] = temp

#Restaurar Equipo como el indice 
    clasificacion_copia.set_index('Equipo', inplace=True)
    return clasificacion_copia


#Creamos un dataframe de 20 filas y 20 columnas, es una matriz asimetrica
#creamos un indice tanto para las columnas como para las filas para tener el dataframe 
#el objetivo es guardar un registro de partidos
registro_partidos = pd.DataFrame(index=range(20), columns=range(20))
registro_partidos = registro_partidos.applymap(lambda x: (0, 0))
registro_partidos.index = clasificacion.index
registro_partidos.columns = clasificacion.index
    
equipos_campeones = defaultdict(int)
equipos_champions = defaultdict(int)
equipos_europaLeague = defaultdict(int)
equipos_descensos = defaultdict(int)

#a aprtir de este bucle lo que haremos sera repetir una y otra vez las simulaciones de 0 a cualquier numer de simulaciones
for _ in range(0,num_simulaciones):
    #copiamos la clasificacion orginal para hacer tantas simulaiciones como queramos
    #y tambien he hecho una copia en enfremientos
    clasificacion_copia = clasificacion.copy()
    enfrentamientos_copia = enfrentamientos.copy()

    #se ejecuta 20 veces el bucle porque hay 20 equipos
    #gracias a esto analizmos que rival le toca a cada equipo
    #calculamos goles de forma aleatoria  y asignamos los puntos correspondientes de cada equipo
    for equipo in clasificacion.index:
        rivales = enfrentamientos_copia[equipo].dropna()
    
        for rival in rivales:
            #si el rival es none lo que haremos es que continue con el siguiente partido
            #si no es None deberá asignar puntos, goles,etc
            if rival == None:
                continue
                
            goles = asignargoles() 
            #con esto accedemos al registro de los goles tanto del local como del visitante
            registro_partidos.at[equipo , rival] = goles
            resultado_equipos = resultado(goles[0],goles[1])
            puntuacion = asignarpuntos(resultado_equipos)

            #debemos añadir los puntos los partidos jugados goles a favor y goles en contra para el equipo y el rival
            #con esta funcion actualizamos la tabal tanto para el equipo que analizamos como para su rival
            clasificacion_copia.loc[equipo]['Pts.'] += puntuacion[0]
            clasificacion_copia.loc[equipo]['PJ'] += 1
            clasificacion_copia.loc[equipo]['GF'] += goles[0]
            clasificacion_copia.loc[equipo]['GC'] += goles[1]
            clasificacion_copia.loc[equipo]['DG'] += (goles[0] - goles[1])
            
            clasificacion_copia.loc[rival]['Pts.'] += puntuacion[1]
            clasificacion_copia.loc[rival]['PJ'] += 1
            clasificacion_copia.loc[rival]['GF'] += goles[1]
            clasificacion_copia.loc[rival]['GC'] += goles[0]
            clasificacion_copia.loc[rival]['DG'] += (goles[1] - goles[0])
            
            #Esta función de none la emplearemos para que no se dupliquen los partidos
            enfrentamientos_copia.loc[enfrentamientos_copia[rival] == equipo,rival] = None

    #con esta funcion tenemos la clasificacion ordenada por puntos
    clasificacion_copia = clasificacion_copia.sort_values(by='Pts.', ascending=False) 
 
    #con esta funcion una vez tenemos la clasificacion buscamos los equipos empatados a puntos y empleamos criterios de reordeacion
    empatados = equipos_empatados(clasificacion_copia)
    clasificacion_copia = reordenacion_clasificacion(clasificacion_copia, empatados)
 
    #aqui almacenamos al campón en el diccionario de campeones 
    #con esto cada vez que un equipo sale campeon se le suma uno
    campeon = resultado_campeonliga(clasificacion=clasificacion_copia)
    equipos_campeones[campeon] += 1

    champions = resultado_Champions(clasificacion=clasificacion_copia)
    for equipo_champions in champions:
        equipos_champions[equipo_champions] += 1

    europaLeague = resultado_EuropaLeague(clasificacion=clasificacion_copia)
    for equipo_europaLeague in europaLeague:
        equipos_europaLeague[equipo_europaLeague] += 1

    descenso = resultado_Descenso(clasificacion=clasificacion_copia)
    for equipo_descenso in descenso:
        equipos_descensos[equipo_descenso] += 1

  
#Convertimos todo a una cadena de texto y lo dejamos a 2 decimales de redondeo 
#Con esto imprimimos las probabilidades de los equipos
print("Probabilidades de ganar la liga\n")   
for campeon in equipos_campeones:
    print(campeon + '-' + str(round(100*(equipos_campeones[campeon]/num_simulaciones),2)) + "%")

print("Probabilidades de entrar en Champions\n")   
for champions in equipos_champions:
    print(champions + '-' + str(round(100*(equipos_champions[champions]/num_simulaciones),2)) + "%")

print("Probabilidades de entrar en Europa League\n")   
for europaLeague in equipos_europaLeague:
    print(europaLeague + '-' + str(round(100*(equipos_europaLeague[europaLeague]/num_simulaciones),2)) + "%")

print("Probabilidades de descender\n")   
for descenso in equipos_descensos:
    print(descenso + '-' + str(round(100*(equipos_descensos[descenso]/num_simulaciones),2)) + "%")

print (clasificacion_copia)

Probabilidades de ganar la liga

Real Madrid-100.0%
Probabilidades de entrar en Champions

Real Madrid-100.0%
Girona-100.0%
Barcelona-100.0%
Atlético-99.24%
Athletic-0.76%
Probabilidades de entrar en Europa League

Athletic-99.24%
R. Sociedad-41.76%
Betis-57.46%
Villarreal-0.28%
Valencia-0.5%
Atlético-0.76%
Probabilidades de descender

Almería-100.0%
Granada-100.0%
Cádiz-91.74%
Celta-5.04%
Las Palmas-0.4%
Rayo-1.42%
Mallorca-1.4%
             PJ  PG  PE  PP  GF  GC  DG  Pts.
Equipo                                       
Real Madrid  38  28   6   1  83  23  60    97
Barcelona    38  23   7   5  80  47  33    83
Girona       38  23   6   6  79  49  30    78
Atlético     38  22   4   9  67  47  20    71
Athletic     38  17  11   7  61  43  18    63
R. Sociedad  38  14  12   9  53  38  15    61
Betis        38  14  13   8  50  46   4    58
Villarreal   38  13   9  13  63  63   0    54
Valencia     38  13   9  13  43  49  -6    51
Getafe       38  10  13  12  50  54  -4    49
Alavés       3

In [3]:
##JORNADA 28

import pandas as pd
import random
#diccionarios
from collections import defaultdict

#Con este excel se carga la clasificación cargada
#Y también caragremos la taba de los enfrentamientos que le quedan a cada equipo
#Con header=0 conseguimos que la primera fila la tome como una fila primcipal
#el index lo ponemos para que se tenga en cuenta en en el excel que los equipos son lo que manda
clasificacion = pd.read_excel('Base de datos Montecarlo.xlsx', sheet_name='clasificacion28', header=0, index_col=0)
enfrentamientos = pd.read_excel('Base de datos Montecarlo.xlsx', sheet_name='enfrentamientos28', header=0)

#Definimos el numero de simulaciones que vamos a realizar
num_simulaciones = 10000
#Añadimos un sistema de puntos para poder implementarlo en el futuro a cualquier deporte
#El orden que sigue es V,E,D
sistema_puntos = [3, 1, 0]

#Implementamops una funcion que por cada partido nos devuleva un resultado aleatorio 
# la funcion dice que si los goles local>visitante gana local y viceversa y sino se devuelve empate
def resultado(goles_local,goles_visitante):
    resultados = ['V', 'E', 'D']
    if goles_local > goles_visitante:
        return resultados[0]
    elif goles_visitante > goles_local:
        return resultados[2]
    else:
        return resultados[1]
    
#Una funcion que asigne puntos en caso de V,D,E, nos fijamos en la raid
# En esta función asiganmos 3 puntos para el equipo que gana y 0 para su rival
def asignarpuntos(resultado):
    if resultado == 'V':
        return (sistema_puntos[0],sistema_puntos[2])
    elif resultado == 'E':
        return (sistema_puntos[1],sistema_puntos[1]) 
    else:
        return (sistema_puntos[ 2],sistema_puntos[0])

#le asignamos los puntos que obtienen en los partidos a la clasificación
#Con esta entrada definimos el campeon de liga de 100000 simulaciones 
#Nos quedaremos siempre con el primero gracias a la fumcion nlargest
def resultado_campeonliga(clasificacion):
    return clasificacion.nlargest(1,'Pts.').index[0]

def resultado_Champions(clasificacion):
    return clasificacion.nlargest(4,'Pts.').index

def resultado_EuropaLeague(clasificacion):
    return clasificacion.nlargest(6,'Pts.').index[4:6]

def resultado_Descenso(clasificacion):
    return clasificacion.nsmallest(3,'Pts.').index
    
#Esta función nos devolverá un numero de goles, poniendo maximo 5 goles en local y visitante
def asignargoles():
    goles_local = random.choice(range(0,5))
    goles_visitante = random.choice(range(0,5))
    
    return (goles_local,goles_visitante)
    
#con la funcion .loc loraremos que nos devuelva primero la informacion de la fila
#con dg1 accedemos a la informacion del equipo 1 y la diferencia de goles que hay
def desempate_golaverage(nombre_equipo1, nombre_equipo2, clasificacion_actual):
    dg1 = clasificacion_actual.loc[nombre_equipo1]['DG']
    dg2 = clasificacion_actual.loc[nombre_equipo2]['DG']
#esta función devuelve el equipo que tenga mas diferencia de goles    
    if dg1 > dg2:
        return nombre_equipo1
    elif dg2 > dg1:
        return nombre_equipo2
    else:
        goles_afavor1 = clasificacion_actual.loc[nombre_equipo1]['GF']
        goles_afavor2 = clasificacion_actual.loc[nombre_equipo2]['GF']
 #esta funcion en caso de que hayan empatado a todo, tendrá prioridad el equipo con mas goles a favor       
        if goles_afavor1 > goles_afavor2:
            return nombre_equipo1
        else:
            return nombre_equipo2
    
#esta función determina un desempate en caso de que haya un empate a puntos y tendremos que acceder a la clasificacion 
#devolvera el nombre de equipo 1 o equipo 2
def desempate_directo(nombre_equipo1, nombre_equipo2, clasificacion_actual):

    #guardamos con esto en resultado tanto para la ida como la vuelta
    resultado1 = registro_partidos.at[nombre_equipo1, nombre_equipo2]
    resultado2 = registro_partidos.at[nombre_equipo2, nombre_equipo1]

    #es la suma del resultado 1 y resultado 2 para equipo 1 y 2
    resultado_global_equipo1 = resultado1[0] + resultado2[1]
    resultado_global_equipo2 = resultado2[0] + resultado1[1]

    #si resultado 1 > resultado 2 se devuelve el nombre del equipo 1 y viceversa
    #y si es igual el resultado tendremos que pasar a la siguiente linea  de desempate por gol average
    if resultado_global_equipo1 > resultado_global_equipo2:
        return nombre_equipo1
    elif resultado_global_equipo2 > resultado_global_equipo1:
        return nombre_equipo2
    else:
        return desempate_golaverage(nombre_equipo1,nombre_equipo2, clasificacion_actual)

 #esta funcion es para los equipos empatados a puntos
def equipos_empatados(clasificacion):
    equipos_empatados = []
    
#el primer equipo se va a comparar con los demas equipos, si los pts del equipo 1 son = al equipo 2 se añade a la lista
    for idx, equipo1 in enumerate (clasificacion.index):
#comparar equipo 1 con los que estan debajo de la clasificacion
        for equipo2 in clasificacion.index[idx+1:]:
#comprobacion para ver si tiene el mismo numero de puntos
            if clasificacion.loc[equipo1, 'Pts.'] == clasificacion.loc[equipo2, 'Pts.']:
                equipos_empatados.append((equipo1, equipo2))
    return equipos_empatados

#con esta funcion reordenamos la clasificacion con los cirterios de reordenacion
def reordenacion_clasificacion(clasificacion_copia, empatados):
#Resetear el indice y guardar la columna equipo
    clasificacion_copia['Equipo'] = clasificacion_copia.index
#Desempatar y reordenar la clasificacion copia
    for equipo1, equipo2 in empatados:
        ganador = desempate_directo(equipo1, equipo2, clasificacion_actual=clasificacion_copia)
        if ganador == equipo1:
            temp = clasificacion_copia.loc[equipo2].copy()
            clasificacion_copia.loc[equipo2] = clasificacion_copia.loc[equipo1]
            clasificacion_copia.loc[equipo1] = temp
        elif ganador == equipo2:
            temp = clasificacion_copia.loc[equipo1].copy()
            clasificacion_copia.loc[equipo1] = clasificacion_copia.loc[equipo2]
            clasificacion_copia.loc[equipo2] = temp

#Restaurar Equipo como el indice 
    clasificacion_copia.set_index('Equipo', inplace=True)
    return clasificacion_copia


#Creamos un dataframe de 20 filas y 20 columnas, es una matriz asimetrica
#creamos un indice tanto para las columnas como para las filas para tener el dataframe 
#el objetivo es guardar un registro de partidos
registro_partidos = pd.DataFrame(index=range(20), columns=range(20))
registro_partidos = registro_partidos.applymap(lambda x: (0, 0))
registro_partidos.index = clasificacion.index
registro_partidos.columns = clasificacion.index
    
equipos_campeones = defaultdict(int)
equipos_champions = defaultdict(int)
equipos_europaLeague = defaultdict(int)
equipos_descensos = defaultdict(int)

#a aprtir de este bucle lo que haremos sera repetir una y otra vez las simulaciones de 0 a cualquier numer de simulaciones
for _ in range(0,num_simulaciones):
    #copiamos la clasificacion orginal para hacer tantas simulaiciones como queramos
    #y tambien he hecho una copia en enfremientos
    clasificacion_copia = clasificacion.copy()
    enfrentamientos_copia = enfrentamientos.copy()

    #se ejecuta 20 veces el bucle porque hay 20 equipos
    #gracias a esto analizmos que rival le toca a cada equipo
    #calculamos goles de forma aleatoria  y asignamos los puntos correspondientes de cada equipo
    for equipo in clasificacion.index:
        rivales = enfrentamientos_copia[equipo].dropna()
    
        for rival in rivales:
            #si el rival es none lo que haremos es que continue con el siguiente partido
            #si no es None deberá asignar puntos, goles,etc
            if rival == None:
                continue
                
            goles = asignargoles() 
            #con esto accedemos al registro de los goles tanto del local como del visitante
            registro_partidos.at[equipo , rival] = goles
            resultado_equipos = resultado(goles[0],goles[1])
            puntuacion = asignarpuntos(resultado_equipos)

            #debemos añadir los puntos los partidos jugados goles a favor y goles en contra para el equipo y el rival
            #con esta funcion actualizamos la tabal tanto para el equipo que analizamos como para su rival
            clasificacion_copia.loc[equipo]['Pts.'] += puntuacion[0]
            clasificacion_copia.loc[equipo]['PJ'] += 1
            clasificacion_copia.loc[equipo]['GF'] += goles[0]
            clasificacion_copia.loc[equipo]['GC'] += goles[1]
            clasificacion_copia.loc[equipo]['DG'] += (goles[0] - goles[1])
            
            clasificacion_copia.loc[rival]['Pts.'] += puntuacion[1]
            clasificacion_copia.loc[rival]['PJ'] += 1
            clasificacion_copia.loc[rival]['GF'] += goles[1]
            clasificacion_copia.loc[rival]['GC'] += goles[0]
            clasificacion_copia.loc[rival]['DG'] += (goles[1] - goles[0])
            
            #Esta función de none la emplearemos para que no se dupliquen los partidos
            enfrentamientos_copia.loc[enfrentamientos_copia[rival] == equipo,rival] = None

    #con esta funcion tenemos la clasificacion ordenada por puntos
    clasificacion_copia = clasificacion_copia.sort_values(by='Pts.', ascending=False) 
 
    #con esta funcion una vez tenemos la clasificacion buscamos los equipos empatados a puntos y empleamos criterios de reordeacion
    empatados = equipos_empatados(clasificacion_copia)
    clasificacion_copia = reordenacion_clasificacion(clasificacion_copia, empatados)
 
    #aqui almacenamos al campón en el diccionario de campeones 
    #con esto cada vez que un equipo sale campeon se le suma uno
    campeon = resultado_campeonliga(clasificacion=clasificacion_copia)
    equipos_campeones[campeon] += 1

    champions = resultado_Champions(clasificacion=clasificacion_copia)
    for equipo_champions in champions:
        equipos_champions[equipo_champions] += 1

    europaLeague = resultado_EuropaLeague(clasificacion=clasificacion_copia)
    for equipo_europaLeague in europaLeague:
        equipos_europaLeague[equipo_europaLeague] += 1

    descenso = resultado_Descenso(clasificacion=clasificacion_copia)
    for equipo_descenso in descenso:
        equipos_descensos[equipo_descenso] += 1

  
#Convertimos todo a una cadena de texto y lo dejamos a 2 decimales de redondeo 
#Con esto imprimimos las probabilidades de los equipos
print("Probabilidades de ganar la liga\n")   
for campeon in equipos_campeones:
    print(campeon + '-' + str(round(100*(equipos_campeones[campeon]/num_simulaciones),2)) + "%")

print("Probabilidades de entrar en Champions\n")   
for champions in equipos_champions:
    print(champions + '-' + str(round(100*(equipos_champions[champions]/num_simulaciones),2)) + "%")

print("Probabilidades de entrar en Europa League\n")   
for europaLeague in equipos_europaLeague:
    print(europaLeague + '-' + str(round(100*(equipos_europaLeague[europaLeague]/num_simulaciones),2)) + "%")

print("Probabilidades de descender\n")   
for descenso in equipos_descensos:
    print(descenso + '-' + str(round(100*(equipos_descensos[descenso]/num_simulaciones),2)) + "%")

print (clasificacion_copia)

Probabilidades de ganar la liga

Real Madrid-77.74%
Girona-11.93%
Barcelona-9.76%
Atlético-0.42%
Athletic-0.15%
Probabilidades de entrar en Champions

Real Madrid-99.96%
Girona-97.48%
Barcelona-94.84%
Atlético-60.57%
Athletic-44.17%
Betis-0.58%
R. Sociedad-1.25%
Valencia-1.11%
Osasuna-0.02%
Getafe-0.01%
Las Palmas-0.01%
Probabilidades de entrar en Europa League

Las Palmas-4.17%
Athletic-53.13%
Valencia-32.5%
Atlético-38.28%
R. Sociedad-30.93%
Betis-26.68%
Osasuna-3.1%
Barcelona-5.14%
Girona-2.52%
Getafe-1.65%
Villarreal-1.64%
Alavés-0.2%
Real Madrid-0.04%
Sevilla-0.01%
Rayo-0.01%
Probabilidades de descender

Almería-99.59%
Granada-96.01%
Celta-26.55%
Cádiz-49.98%
Rayo-12.72%
Sevilla-6.07%
Mallorca-8.22%
Alavés-0.57%
Getafe-0.11%
Osasuna-0.06%
Las Palmas-0.03%
Villarreal-0.09%
             PJ  PG  PE  PP  GF  GC  DG  Pts.
Equipo                                       
Real Madrid  38  21   6   1  83  33  50    86
Girona       38  19   5   4  80  47  33    78
Barcelona    38  18   7   3 